In [1]:
import torch as tr
import numpy as np
from CSW import *

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# match human data
- to match human i need to prop one sentence at a time. otherwise the network will do well on later sentences (because after observing one transition the next are determined)
- backprop for each sample, flush at end of story

In [2]:
task = CSWTask(graph_pr=1.0)

In [3]:
# task
pr = 1.0
taskL = [CSWTask(pr),CSWTask(1-pr)]

eval_pathL = [
  [10,0,1,4,5],       
  [11,0,1,3,5],
  ]

xeval = taskL[0].format_Xeval(eval_pathL)
xeval = tr.tensor(xeval)

In [5]:
# condition
block_len0 = 40
bpmode='story'
# nnets
nnets = 10
## params
stsize = 40
learn_rate = 0.05
neps = 200

## train setup
softmax = lambda ulog: tr.softmax(ulog,-1)
lossop = tr.nn.CrossEntropyLoss()
acc = -np.ones([nnets,neps])
# eval array
tdim,sm_dim=5,12
ysm = -np.ones([nnets,2,neps,tdim,sm_dim])

## loop over seeds
for seed in range(nnets):
  lossL = []
  # init net
  net = CSWNet(stsize,seed)
  optiop = tr.optim.Adam(net.parameters(), lr=learn_rate)
  ## train loop
  task_int = 0
  block_len = block_len0 
  for ep in range(neps):
    if ep >= 160:
      block_len = 1
    # select graph
    if ep%block_len==0:
      task_int = (task_int+1)%2
      task = taskL[task_int]
      filler_id = 10+task_int 
    # forward prop  
    path = task.sample_path()
    xtrain,ytrain = task.dataset_onestory_with_marker(path=path,filler_id=filler_id,depth=1)
    yh = net(xtrain) # (time,smunits)
    yh_sm = softmax(yh).detach().numpy()
    yt = ytrain.detach().numpy().squeeze()
    acc[seed,ep] = np.mean(np.equal(np.argmax(yh_sm[3:4],1),yt[3:4]))
    # eval
    for idx,xev in enumerate(xeval):
      ysm_t = softmax(net(xev)).detach().numpy()
      ysm[seed,idx,ep] = ysm_t
    if bpmode=='scene':
      loss = 0
      for tstep in range(len(xtrain)):
        loss = lossop(yh[tstep].unsqueeze(0),ytrain[tstep])
        optiop.zero_grad()
        loss.backward(retain_graph=True)
        optiop.step()
    elif bpmode=='story':
      loss = 0
      for tstep in range(len(xtrain)):
        loss += lossop(yh[tstep].unsqueeze(0),ytrain[tstep])
      optiop.zero_grad()
      loss.backward(retain_graph=True)
      optiop.step()
    lossL.append(loss)

fpath = 'eval_data/matchSEM/'
np.save(fpath+'ysm-blocklen%i-bpmode_%s'%(block_len0,bpmode),ysm)
np.save(fpath+'acc-blocklen%i-bpmode_%s'%(block_len0,bpmode),acc)